In [1]:
from swf import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit,lsq_linear,brute
from scipy.sparse.linalg import LinearOperator
from scipy.stats import norm
import plotly.express as px
import pandas as pd

In [2]:
octvertices = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]])
octfaces = np.array([[1,2,4],[1,3,4],[3,0,4],[0,2,4],[1,3,5],[3,0,5],[0,2,5],[2,1,5]])

layout704=np.array([[1,0,np.pi/2],[1,np.pi/6,np.pi/2],[1,-np.pi/6,np.pi/2],[1,np.pi/2,np.pi/2],[1,-np.pi/2,np.pi/2],[1,3*np.pi/4,np.pi/2],[1,-3*np.pi/4,np.pi/2],[1,np.pi/4,np.pi/4],[1,-np.pi/4,np.pi/4],[1,3*np.pi/4,np.pi/4],[1,-3*np.pi/4,np.pi/4]])
vertices704 = np.apply_along_axis(lambda x: toCartesian(x),1,layout704)
faces704 = np.array([[6,4,10],[10,4,8],[8,4,2],[8,2,0],[8,7,0],[7,0,1],[7,1,3],[7,3,9],[9,3,5],[10,7,9],[10,8,7],[10,6,5],[10,9,5]])

vertices301 = np.array([[1,0,0],[-1/2,np.sqrt(3)/2,0],[-1/2,-np.sqrt(3)/2,0],[0,0,1]])
faces301 = np.array([[0,1,3],[1,2,3],[2,0,3]])

In [3]:
def optimizeOCT(coeffs):
    print('.',end='')
    ALPHA, BETA = coeffs
    GAMMA = (1-2*(ALPHA+BETA))/4
    if checkCoeffRelations(ALPHA,BETA,GAMMA):
        mesh = Trimesh(octvertices,octfaces,ALPHA=ALPHA,BETA=BETA,GAMMA=GAMMA)
        model = SWF(mesh, n=3)
        return cost(model,1,1)
    else:
        return 100

In [4]:
A = LinearOperator((1,2),optimizeOCT)
A

.

<1x2 _CustomLinearOperator with dtype=float64>

In [14]:
aDist = norm(loc=0.62, scale=0.005)
bDist = norm(loc=-0.01, scale=0.005)
aRVS = aDist.rvs(size=1200).reshape((-1,1))
bRVS = bDist.rvs(size=1200).reshape((-1,1))
v = np.hstack((aRVS,bRVS))
b = np.array([])
for i in range(1200):
    b = np.append(b,A.matvec(v[i]))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [15]:
df = pd.DataFrame(np.hstack((v,b.reshape(-1,1))),columns=['a','b','cost'])
fig = px.scatter_3d(df[df.cost<0.0014], x="a", y="b", z='cost',color='cost')
fig.show()

In [16]:
df[df.cost ==df['cost'].min()]

,a,b,cost
239,0.608872,-0.014081,0.001298


In [8]:
def optimize704(coeffs):
    print('.',end='')
    ALPHA, BETA = coeffs
    GAMMA = (1-2*(ALPHA+BETA))/4
    if checkCoeffRelations(ALPHA,BETA,GAMMA):
        mesh = Trimesh(vertices704,faces704,ALPHA=ALPHA,BETA=BETA,GAMMA=GAMMA)
        model = SWF(mesh, n=3)
        return cost(model,1,1)
    else:
        return 100

In [9]:
B = LinearOperator((1,2),optimize704)
B

.

<1x2 _CustomLinearOperator with dtype=float64>

In [10]:
aDist = norm(loc=0.55, scale=0.01)
bDist = norm(loc=0.03, scale=0.01)
aRVS = aDist.rvs(size=1200).reshape((-1,1))
bRVS = bDist.rvs(size=1200).reshape((-1,1))
v = np.hstack((aRVS,bRVS))
b = np.array([])
for i in range(1200):
    b = np.append(b,B.matvec(v[i]))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [13]:
df2 = pd.DataFrame(np.hstack((v,b.reshape(-1,1))),columns=['a','b','cost'])
fig = px.scatter_3d(df2[df2.cost<0.0086], x="a", y="b", z='cost',color='cost')
fig.show()

In [12]:
df2[df2.cost ==df2['cost'].min()]

,a,b,cost
534,0.545834,0.037434,0.00849


In [18]:
def optimize301(coeffs):
    print('.',end='')
    ALPHA, BETA = coeffs
    GAMMA = (1-2*(ALPHA+BETA))/4
    if checkCoeffRelations(ALPHA,BETA,GAMMA):
        mesh = Trimesh(vertices301,faces301,ALPHA=ALPHA,BETA=BETA,GAMMA=GAMMA)
        model = SWF(mesh, n=3)
        return cost(model,1,1)
    else:
        return 100

In [19]:
C = LinearOperator((1,2),optimize301)
C

.

<1x2 _CustomLinearOperator with dtype=float64>

In [27]:
aDist = norm(loc=0.5, scale=0.5)
bDist = norm(loc=0, scale=0.5)
aRVS = aDist.rvs(size=1200).reshape((-1,1))
bRVS = bDist.rvs(size=1200).reshape((-1,1))
v = np.hstack((aRVS,bRVS))
b = np.array([])
for i in range(1200):
    b = np.append(b,C.matvec(v[i]))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [28]:
df3 = pd.DataFrame(np.hstack((v,b.reshape(-1,1))),columns=['a','b','cost'])
fig = px.scatter_3d(df3, x="a", y="b", z='cost',color='cost')
fig.show()

In [29]:
df3[df3.cost ==df3['cost'].min()]

,a,b,cost
668,0.626671,0.012229,0.021653


In [30]:
df3

,a,b,cost
0,1.328156,-0.042593,27.789137
1,0.818692,-0.760479,NaN
2,1.674037,0.431349,409.235744
3,0.159810,0.168026,NaN
4,0.786213,-0.309085,NaN
...,...,...,...
1195,0.591370,-0.293599,NaN
1196,0.947434,0.101505,3.929304
1197,0.805775,-0.897928,NaN
1198,-0.348639,-0.357945,NaN
